# Multiagent Pattern - Multiagent Collaboration

<img src="../img/multi_agent_pattern.png" alt="Alt text" width="500"/>

We follow the concept of crewai with our implementation. We create a crew which is a structured group of AI agents that collaborate to complete complex tasks together. 

Each agent has a distinct role. We declare dependencies between agents. This way the output of one agent, can be the input for another agent.


In [1]:
#%pip install -r requirements.txt
%pip install --upgrade pip
%pip install jupyter ipykernel
%%python3 -m ipykernel install --user --name=python310 --display-name "Python 3.10"


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


UsageError: Line magic function `%%python3` not found.


In [2]:
import sys
print("Python version:", sys.version)

Python version: 3.10.14 (main, Mar 19 2024, 21:46:16) [Clang 15.0.0 (clang-1500.1.0.2.5)]


In [3]:
%pip install openai python-dotenv graphviz colorama

Note: you may need to restart the kernel to use updated packages.


In [4]:
import openai
import dotenv
import graphviz
import colorama
print("✅ All dependencies installed successfully!")

✅ All dependencies installed successfully!


In [5]:
import sys
import os

project_root = os.getcwd()
src_path = os.path.join(project_root, "src")
if src_path not in sys.path:
    sys.path.append(src_path)

print("Updated sys.path:", sys.path)

Updated sys.path: ['/usr/local/Cellar/python@3.10/3.10.14/Frameworks/Python.framework/Versions/3.10/lib/python310.zip', '/usr/local/Cellar/python@3.10/3.10.14/Frameworks/Python.framework/Versions/3.10/lib/python3.10', '/usr/local/Cellar/python@3.10/3.10.14/Frameworks/Python.framework/Versions/3.10/lib/python3.10/lib-dynload', '', '/Users/glin/Documents/GitHub/M-LLAP/venv2/lib/python3.10/site-packages', '/Users/glin/Documents/GitHub/M-LLAP/design_patterns/src']


## The Agent Class

First of all, we need an **Agent Class**. This class implements an Agent, and internally it implements the ReAct technique.

In [6]:
import sys
import os
os.environ["OPENAI_API_KEY"]=" "
print("API Key set:", os.getenv("OPENAI_API_KEY") is not None)


API Key set: True


In [7]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from src.multi_agent.agent import Agent
print("✅ Agent imported successfully!")

✅ Agent imported successfully!


Let's create some example agent, to see how it works.

You can also associate tools with the agent. Let's create a tool for writing some string into a CSV.

In [8]:
from src.tool_agent.tool import tool

In [9]:
@tool
def write_str_to_txt(string_data: str, txt_filename: str):
    """
    Writes a string to a txt file.

    This function takes a string and writes it to a text file. If the file already exists, 
    it will be overwritten with the new data.

    Args:
        string_data (str): The string containing the data to be written to the file.
        txt_filename (str): The name of the text file to which the data should be written.
    """
    # Write the string data to the text file
    with open(txt_filename, mode='w', encoding='utf-8') as file:
        file.write(string_data)

    print(f"Data successfully written to {txt_filename}")

In [16]:
agent_tool_example = Agent(
    name="Writer Agent",
    backstory="You are a language model specialised in writing text into .txt files",
    task_description="Write the string 'This is a Tool Agent' into './tool_agent_example.txt'",
    task_expected_output="A .txt file containing the given string",
    tools=write_str_to_txt,
)

In [17]:
agent_tool_example.run()


Thought: I need to write the string 'This is a Tool Agent' into a text file named 'tool_agent_example.txt'.

Using Tool: write_str_to_txt

Tool call dict: 
{'name': 'write_str_to_txt', 'arguments': {'string_data': 'This is a Tool Agent', 'txt_filename': 'tool_agent_example.txt'}, 'id': 0}
Data successfully written to tool_agent_example.txt

Tool result: 
None

Observations: {0: None}


"The string 'This is a Tool Agent' has been successfully written to 'tool_agent_example.txt'."

Let's define two agents now.

In [10]:
from src.multi_agent.crew import Crew

In [12]:
with Crew() as crew:
    # ---- Member & Time Setup Agent ---- #
    MT_Agent = Agent(
        name="Member & Time Setup Agent",
        backstory="You track family members' arrivals and ensure accurate scheduling.",
        task_description="Set up arrival times, locations, and travel durations for all family members.",
        task_expected_output="Structured schedule ensuring all members arrive on time for dinner. - Sarah(Mom):Host,athome, - James(Dad):LandsatBOS1:00PMfromSF, - Emily(Sister):LandsatBOS2:30PMfromChicago - Michael(Brother):Driving,arrives3:00PMfromNY - Grandma:NeedspickupfromsuburbanBoston"
    )
    
    # ---- Requirement Setup Agent ---- #
    RS_Agent = Agent(
        name="Requirement Setup Agent",
        backstory="You manage cooking schedules and key logistical needs.",
        task_description="Schedule turkey and side dish preparation while ensuring someone stays home for supervision.",
        task_expected_output="Optimized cooking schedule aligning with dinner timing. cooking requirements:  -Turkey:4hourscookingtime, - Sidedishes:2hourspreparation, - Someonemuststayhomeduringcooking, "
    )
    
    # ---- Constraint Validation Agent ---- #
    CV_Agent = Agent(
        name="Constraint Validation Agent",
        backstory="You verify all scheduling constraints and ensure compliance.",
        task_description="Validate that all pickups, cooking timelines, and supervision requirements are met.",
        task_expected_output="A conflict-free schedule ensuring all tasks are completed efficiently. - Jamesmustrentcarafterlanding, - Emilyrequiresairportpickup ,- Traveltimes: – HometoBOSAirport:60min – BOSAirporttoGrandma’s:60min – HometoGrandma’s:30min"
    )
    
    # ---- Supervisor Agent ---- #
    SA_Agent = Agent(
        name="Supervisor Agent",
        backstory="You oversee all logistical elements and generate the final dinner preparation report.",
        task_description="Monitor and report on key tasks, including cooking start time, Emily's pickup, and Grandma's pickup.",
        task_expected_output="Comprehensive report detailing dinner preparation logistics and arrivals. key requirement: - Allfamilymembersathomefor6:00PMdinner - Turkeyandsidesreadybydinnertime - Allpickupscompletedwithavailabledrivers - Cookingsupervisionmaintained"
    )

    Writer_agent = Agent(
        name="Writer Agent",
        backstory="You are a language model specialised in writing text into .json files",
        task_description="Write the json response into './p5_output.json'",
        task_expected_output="A .json file containing the given string",
        tools=write_str_to_txt,
    )
    
    # ---- Define Dependencies ---- #
    MT_Agent >> RS_Agent >> CV_Agent >> SA_Agent >> Writer_agent


In [13]:
crew.run()


RUNNING AGENT: Member & Time Setup Agent

Structured schedule ensuring all members arrive on time for dinner. 
- Sarah(Mom): Host, at home
- James(Dad): Lands at BOS 1:00 PM from SF
- Emily(Sister): Lands at BOS 2:30 PM from Chicago
- Michael(Brother): Driving, arrives 3:00 PM from NY
- Grandma: Needs pickup from suburban Boston

RUNNING AGENT: Requirement Setup Agent

To ensure a smooth preparation and timely dinner, here's an optimized cooking schedule:

1. **Turkey Preparation:**
   - Start Time: 11:00 AM
   - End Time: 3:00 PM
   - Cooking Time: 4 hours

2. **Side Dishes Preparation:**
   - Start Time: 1:00 PM
   - End Time: 3:00 PM
   - Preparation Time: 2 hours

**Supervision Schedule:**
- **Sarah (Mom)** will be at home to supervise the entire cooking process from 11:00 AM onwards.

**Logistics:**
- **James (Dad)** arrives at 1:00 PM and can assist with any last-minute preparations or errands.
- **Emily (Sister)** arrives at 2:30 PM and can help with setting the table or other 